# Modeling v2.0

My first set of models didn't perform very well. I only had 4 features (one of which was not very important), so that seems like an area for improvement. Some additional features which could be helpful are:

* A sentiment rating for the review text, rating is as either negative, positive or neutral
* A listing of the top 3 most similiar books, based on a cosine similarity analsyis of each book's text description

For the second feature, I'll use that list of most similiar books by looking at:
* The average rating of each book 
* The average rating assigned to those books by that reader's cluster

__First, reload our data:__

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# Here is our previous model's data, all cleaned and ready to go
data = pd.read_csv('data/model1_data_cleaned.csv', index_col=['user_id','book_id'])
data.head()

rating  user_avg_rating  \
user_id                          book_id                             
000192962b87d560f00b06fdcbd71681 30025791       5              5.0   
0005a08accd53b1e19c52109a1f478cb 59960          0              3.4   
000700ecd5db3a9b0c4e392ed2e4f70b 11790194       5              5.0   
0008931c0cde961e9c802c5a58196d23 500503         5              5.0   
                                 6081685        5              5.0   

                                           book_cluster  \
user_id                          book_id                  
000192962b87d560f00b06fdcbd71681 30025791             0   
0005a08accd53b1e19c52109a1f478cb 59960                4   
000700ecd5db3a9b0c4e392ed2e4f70b 11790194             0   
0008931c0cde961e9c802c5a58196d23 500503               1   
                                 6081685              1   

                                           user_avg_rating_by_cluster  \
user_id                          book_id                                
000192962b87d560f00b06fdcbd71681 30025791                         5.0   
0005a08accd53b1e19c52109a1f478cb 59960                            0.0   
000700ecd5db3a9b0c4e392ed2e4f70b 11790194                         5.0   
0008931c0cde961e9c802c5a58196d23 500503                           5.0   
                                 6081685                          5.0   

                                           book_avg_rating  
user_id                          book_id                    
000192962b87d560f00b06fdcbd71681 30025791             4.19  
0005a08accd53b1e19c52109a1f478cb 59960                4.25  
000700ecd5db3a9b0c4e392ed2e4f70b 11790194             4.04  
0008931c0cde961e9c802c5a58196d23 500503               4.29  
                                 6081685              4.40

## First Feature: Sentiment Rating

For each review, we want to run a sentiment analysis on the review text to assign it a negative, positive or neutral rating.

In [4]:
# Need to reload our review dataframe
reviews = pd.read_csv('data/reviews_step3_output.csv')
reviews.head()

,Unnamed: 0,review_id,user_id,book_id,rating,review_text,year
0,0,66b2ba840f9bd36d6d27f46136fe4772,dc3763cdb9b2cae805882878eebb6a32,18471619,3,Sherlock Holmes and the Vampires of London \n ...,2013
1,1,72f1229aba5a88f9e72f0dcdc007dd22,bafc2d50014200cda7cb2b6acd60cd73,6315584,4,"I've never really liked Spider-Man. I am, howe...",2016
2,2,a75309355f8662caaa5e2c92ab693d3f,bafc2d50014200cda7cb2b6acd60cd73,29847729,4,"A very quick introduction, this is coming out ...",2016
3,3,c3cc5a3e1d6b6c9cf1c044f306c8e752,bafc2d50014200cda7cb2b6acd60cd73,18454118,5,I've been waiting so long for this. I first st...,2014
4,4,cc444be37ab0a42bfb4dd818cb5edd10,bafc2d50014200cda7cb2b6acd60cd73,2239435,4,The only thing more entertaining than this boo...,2013
